In [39]:
#INSTALLING THE KAGGLE API

!pip install kaggle #installing the kaggle API
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
#copying the .json file to the required directory
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json


Download 100%.
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [40]:
#downloading the files
!kaggle datasets download -d thedagger/pokemon-generation-one -p /content/kaggle

pokemon-generation-one.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
#extracting the training zip file
import zipfile

local_zip = '/content/kaggle/pokemon-generation-one.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/utkarsh')
zip_ref.close()

In [42]:
os.listdir()

['.config', '.kaggle', 'kaggle', 'utkarsh', 'adc.json', 'sample_data']

In [0]:
#importing required libraries

#image precessing
import cv2
#file handling
import os
#obvious
import numpy as np
import pandas as pd
#library to know the progress of for loops
from tqdm import tqdm

In [0]:
#the path where the required files are
base_dir = '/content/utkarsh'
train_dir = os.path.join(base_dir, 'dataset')
# test_dir = os.path.join(base_dir,'test')

In [45]:
os.listdir(train_dir)

['Electabuzz',
 'Exeggcute',
 'Shellder',
 'Paras',
 'Poliwhirl',
 'Moltres',
 'Slowbro',
 'Jolteon',
 'Ninetales',
 'Venonat',
 'Drowzee',
 'Poliwrath',
 'Golbat',
 'Machamp',
 'Psyduck',
 'Gengar',
 'Zubat',
 'NidoranΓÖé',
 'Golem',
 'Ditto',
 'Horsea',
 'Raichu',
 'Nidorino',
 'Butterfree',
 'Fearow',
 'Krabby',
 'Charmeleon',
 'Zapdos',
 'Jynx',
 'Weepinbell',
 'Ivysaur',
 'Starmie',
 'Venomoth',
 'Sandslash',
 'Electrode',
 'Doduo',
 'Nidoqueen',
 'Mewtwo',
 'Dratini',
 'Voltorb',
 'Metapod',
 'Gastly',
 'Raticate',
 'Clefairy',
 'Hypno',
 'Articuno',
 'Flareon',
 'Meowth',
 'Persian',
 'Machop',
 'Aerodactyl',
 'Ponyta',
 'Jigglypuff',
 'Victreebel',
 'Lapras',
 'Mankey',
 'Exeggutor',
 'Tauros',
 'Spearow',
 'Oddish',
 'Mew',
 'Abra',
 'Graveler',
 'Rapidash',
 'Kakuna',
 'Kingler',
 'Geodude',
 'Farfetchd',
 'Nidorina',
 'Dugtrio',
 'Magmar',
 'Gloom',
 'Sandshrew',
 'Dragonite',
 'Bulbasaur',
 'Arbok',
 'Kadabra',
 'Ekans',
 'Kabutops',
 'Tentacool',
 'Pidgeot',
 'Kangaskhan',

In [0]:
#loading images and returning an array containing the features
IMG_SIZE = 50
def load_images_from_folder(folder):
    images = []
    for folders in tqdm(os.listdir(folder)):
      for filename in os.listdir(os.path.join(folder,folders)):
#           print(folders,filename)
          if(filename==".DS_Store"):
            continue
          if(filename.split(".")[1]=="jpg"):
            img = cv2.imread(os.path.join(folder,os.path.join(folders,filename)))
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            img = img/255
            if img is not None:
              images.append([np.array(img),folders])
    return images

In [47]:
train1 = load_images_from_folder("/content/utkarsh/dataset")


100%|██████████| 151/151 [01:16<00:00,  1.83it/s]


In [0]:
len(train1)
x = []
y = []
for i in train1:
  x.append(i[0])
  y.append(i[1])

In [0]:
x = np.array(x)


In [50]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train1 = encoder.fit_transform(y)
len(encoder.classes_)

149

In [0]:
#splitting the data into training and testing data
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(train1,y,random_state=0,test_size=0.2)

In [0]:
#importing keras and required layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

In [0]:
nClasses = 149

In [0]:
#constructing the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50,50,3)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
 
model.add(Conv2D(96, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(96, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nClasses, activation='softmax'))

In [55]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 50, 50, 32)        896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 50, 50, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 25, 25, 64)        18496     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 25, 25, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 64)        0         
__________

In [56]:
model.fit(x,y_train1,epochs =100,batch_size = 50)

Epoch 1/100
10118/10118 [==============================] - 9s 909us/step - loss: 4.9082 - acc: 0.0243
Epoch 2/100
10118/10118 [==============================] - 8s 797us/step - loss: 4.5175 - acc: 0.0649
Epoch 3/100
10118/10118 [==============================] - 8s 800us/step - loss: 4.2083 - acc: 0.0989
Epoch 4/100
10118/10118 [==============================] - 8s 799us/step - loss: 3.8173 - acc: 0.1547
Epoch 5/100
10118/10118 [==============================] - 8s 802us/step - loss: 3.3962 - acc: 0.2273
Epoch 6/100
10118/10118 [==============================] - 8s 801us/step - loss: 3.0267 - acc: 0.2902
Epoch 7/100
10118/10118 [==============================] - 8s 797us/step - loss: 2.7099 - acc: 0.3471
Epoch 8/100
10118/10118 [==============================] - 8s 799us/step - loss: 2.4032 - acc: 0.4086
Epoch 9/100
10118/10118 [==============================] - 8s 798us/step - loss: 2.1371 - acc: 0.4619
Epoch 10/100
10118/10118 [==============================] - 8s 802us/step - loss: 

In [58]:
loss2, acc2 = model.evaluate(x,y_train1)

10118/10118 [==============================] - 3s 318us/step


In [59]:
print(acc2)

0.9814192528167622
